In [1]:
import requests
import re
import pandas as pd
import numpy as np
import os
from time import perf_counter
from functools import wraps
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime
import logging

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler = logging.FileHandler('logs/app.log', mode='a')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

### Functions, wrappers and request headers

In [23]:
headers = {'Accept': 'text/html',
           'Accept-Language': 'en-US',
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/605.1.15 Version/13.0.4',
           'Referer': 'http://www.google.com/'}

def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = perf_counter()
        result = func(*args, **kwargs)
        end_time = perf_counter()
        total_time = end_time - start_time
        logger.info(f'Function {func.__name__} Took {total_time:.4f} seconds - {args} {kwargs}')
        return result
    return timeit_wrapper

@timeit
def get_job_links(keyword, start_page, pages):    
    '''Every linkedin job search page carries 25 jobs'''
    def custom_selector(tag):
        return tag.name == "a" and tag.has_attr("href") and keyword in tag.get('href')
    title = re.sub(' ', '%20', keyword)
    keyword = re.sub(' ', '-', keyword) # This is used inside custom_selector's scope   
    logger.info(f'Searching for {keyword}')
    job_links = []
    position = start_page
    # currentJobId = None
    try:
        for page in tqdm(range(pages)):
            # url = f"https://www.linkedin.com/jobs/search/?currentJobId=3693533935&distance=25&geoId=102454443&keywords={title}&origin=JOB_SEARCH_PAGE_KEYWORD_HISTORY&refresh=true&start={position}"
            # url = f"https://www.linkedin.com/jobs/search/?currentJobId=3693533935&distance=25&geoId=102454443&keywords={title}&currentJobId=3415227738&position=1&pageNum=0&start={position}" 
            # if not currentJobId:
            #     url = f"https://www.linkedin.com/jobs/search/?geoId=102454443&keywords={title}&location=Singapore&start={position}"
            # else:
            #     url = f"https://www.linkedin.com/jobs/search/?currentJobId={currentJobId}&geoId=102454443&keywords={title}&location=Singapore&start={position}"
            url = f"https://www.linkedin.com/jobs/search/?geoId=102454443&keywords={title}&location=Singapore&start={position}"
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.text,'html.parser')
            tags = soup.find_all(custom_selector)
            for tag in tags:
                link = tag.get('href')
                job_links.append(link)
            # Get the last jobid to start on the next page
            # currentJobId = re.findall('-([0-9]{6,})\?', link)[0]
            position += 25
        logger.indo('Retrieved links: ', len(job_links), '\n')
    except Exception as e:
        logger.error(f'Error at page {page}, {e}')
    finally:
        return job_links
    
def get_job_info(url, return_soup=False):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text,'html.parser')
    info = {}

    # Page title
    title = soup.find('title')
    if title:
        info['company'] = title.text.split(' hiring')[0]

    # Job title
    job_title = soup.find('h1')
    if job_title:
        info['job_title'] = job_title.text

    # Job level, type (full time etc), sector
    criteria = soup.find_all('span', class_="description__job-criteria-text description__job-criteria-text--criteria")
    if criteria:
        criteria = [x.text.strip(' \n') for x in criteria]
        try:
            info['level'] = criteria[0]
            info['job_type'] = criteria[1]
            info['industry1'] = criteria[2]
            info['industry2'] = criteria[3]
        except Exception as e:
            logger.error(f'{e, criteria, url}')

    # Job scope and requirements
    descriptions = soup.find(class_ = "show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden")
    if descriptions:
        # If list form:
        bullets = [li.text for li in descriptions.find_all('li')]
        semicolon_splits = descriptions.text.split(';')
        newline_splits = descriptions.text.split('\n')

        line_by_line_desc = ''
        experience = ''
        spark = ''
        degree = ''

        if len(bullets)>0:
            chosen_method = bullets
        elif len(semicolon_splits) > len(newline_splits):
            logger.info('Fall back to semicolon split')
            chosen_method = semicolon_splits
        else:
            logger.info('Fall back to newline split')
            chosen_method = newline_splits
        
        for desc in chosen_method:
            if 'experience' in desc:
                experience += desc + '\n'
            if ('PySpark' in desc) or ('Spark' in desc):
                spark += desc + '\n'
            if 'degree' in desc:
                degree += desc + '\n'
            line_by_line_desc += desc + '\n'

        info['descriptions'] = line_by_line_desc
        info['degree'] = degree
        info['experience'] = experience   
        info['spark'] = spark 
    else: # Print notification if nothing found
        logger.error(f'Found no description for {url}')

    info['link'] = url            
    
    if return_soup:
        return info, soup
    else:
        return info
    

def process_df(data):
    df = pd.DataFrame.from_dict(data, orient='index')
    wNulls = len(df)
    df = df[~(df['company'].isnull() & df['job_title'].isnull() & df['level'].isnull() & df['descriptions'].isnull())]
    logger.info(f'Removed {wNulls - len(df)} empty roles')

    # Deduplication, some jobs are similar but have different link maybe due to their different posting time / reposting
    subset_duplicates = ['company', 'job_title', 'level', 'job_type', 'degree', 'experience', 'spark', 'descriptions', 'industry1']
    wDups = len(df)
    df = df.drop_duplicates(subset=subset_duplicates)
    logger.info(f'Removed {wDups - len(df)} duplicates')

    # Sorting order and values
    df = df[['company', 'job_title', 'level', 'job_type', 'experience', 'spark', 'degree', 'descriptions', 'industry1', 'industry2', 'link']]
    df = df.sort_values(by=['level', 'spark', 'company', 'job_type'],
                        ascending= [True, False, True, True])
    logger.info(f'Extracted {len(df)} number of jobs')

    return df

def append_to_main(main_df_filepath, most_recent_filepaths):
    # Appending to existing dataframe
    subfolder = 'ignore/'
    main_df = pd.read_excel(subfolder+main_df_filepath)
    subset_duplicates = ['company', 'job_title', 'level', 'job_type', 'degree', 'experience', 'spark', 'descriptions', 'industry1']
    original_rows = len(main_df)
    logger.info(f'Original rows: {original_rows}')

    if most_recent_filepaths:
        for filepath in most_recent_filepaths:
            if 'MAIN' not in filepath:
                sub_df = pd.read_excel(subfolder+filepath, index_col=0)
                # sub_df.insert(2, 'Emailed  / Messaged Recruiter', np.nan)
                # sub_df.insert(2, 'Apply', np.nan)
                logger.info(f'Read rows: {len(sub_df)}')
                main_df = pd.concat([main_df, sub_df])
                main_df = main_df.drop_duplicates(subset=subset_duplicates)

    else:
        excel_filepaths = [file for file in os.listdir('ignore') if file.endswith('xlsx')]
        for item in enumerate(excel_filepaths):
            print(item)
        idx = input("Select file index: ")
        sub_df = pd.read_excel(subfolder+excel_filepaths[idx], index_col=0)
        # sub_df.insert(2, 'Emailed  / Messaged Recruiter', np.nan)
        # sub_df.insert(2, 'Apply', np.nan)
        logger.info(f'Read rows: {len(sub_df)}')
        main_df = pd.concat([main_df, sub_df])
        main_df = main_df.drop_duplicates(subset=subset_duplicates)
    
    logger.info(f'Added rows: {len(main_df)-original_rows}')
    
    return main_df

## Input job search keyword

In [4]:
keyword = 'data analyst'
pages = 5

In [5]:
links = get_job_links(keyword, start_page=0, pages=pages)
main = {}
logger.info('Getting job info')
for index, link in tqdm(enumerate(links), total = len(links), dynamic_ncols =True):
    main[index] = get_job_info(link)
# for index, link in enumerate(links):
#     main[index] = get_job_info(link)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 107/107 [02:15<00:00,  1.27s/it]


In [10]:
df = process_df(main)
df.head()

INFO:__main__:Removed 6 empty roles
INFO:__main__:Removed 59 duplicates
INFO:__main__:Extracted 42 number of jobs


,company,job_title,level,job_type,experience,spark,degree,descriptions,industry1,industry2,link
52,Shopee,"Data Analyst - Marketing Analytics, Regional B...",Associate,Full-time,,Develop smart and efficient solutions to repor...,,Provide data and insight support for specific ...,"Analyst, Marketing, and Information Technology","Technology, Information and Internet",https://sg.linkedin.com/jobs/view/data-analyst...
13,HCLTech,Data Analyst (SQL/Snowflake),Associate,Full-time,At least 3 years of data-related working exper...,,Bachelor degree from a recognized tertiary ins...,"Working closely with business end-users, marke...",Information Technology,IT Services and IT Consulting,https://sg.linkedin.com/jobs/view/data-analyst...
40,SHIELD,Data Analyst (Risk),Associate,Full-time,3-5 years of experience as a hands-on analyst...,,,Analysis of rich user and transaction data to ...,Analyst,"Technology, Information and Internet",https://sg.linkedin.com/jobs/view/data-analyst...
2,Starbucks Coffee Singapore,Data Analyst,Associate,Full-time,,,"At least a Bachelor degree in Statistics, Math...",Manage end-to-end data projects; identify issu...,Analyst and Information Technology,Food and Beverage Services,https://sg.linkedin.com/jobs/view/data-analyst...
34,TRITON AI PTE LTD,Shipping Data Analyst,Associate,Full-time,You should have at least 2 years of relevant w...,,"Bachelor’s degree in data science, statistics,...","Perm, Central\nYou should have at least 2 year...",Analyst,"Technology, Information and Internet",https://sg.linkedin.com/jobs/view/shipping-dat...


In [11]:
most_recent_file = f"{keyword}_{datetime.now().strftime('%Y-%m-%d-%H%M')}.xlsx"
df.to_excel(f"ignore/{most_recent_file}", engine='xlsxwriter')

In [24]:
most_recent_filepaths = [file for file in os.listdir('ignore') if datetime.now().strftime('%Y-%m-%d') in file]
most_recent_filepaths

['data analyst_2023-12-22-1425.xlsx', 'data analyst_2023-12-22-1427.xlsx']

In [9]:
# # Building new dataframe
# main_df = pd.DataFrame(columns = ['company', 'job_title', 'level', 'job_type', 'experience', 'spark', 'degree', 'descriptions', 'industry1', 'industry2', 'link'])

In [25]:
main_df = append_to_main('MAIN_2023-12-20.xlsx', most_recent_filepaths)

INFO:__main__:Original rows: 111
INFO:__main__:Read rows: 42
INFO:__main__:Read rows: 42
INFO:__main__:Added rows: 8


In [27]:
if f"MAIN_{datetime.now().strftime('%Y-%m-%d')}.xlsx" in os.listdir('ignore'):
    pass
else:
    main_df.to_excel(f"ignore/MAIN_{datetime.now().strftime('%Y-%m-%d')}.xlsx", engine='xlsxwriter')